### save when iterrupt

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras_text_summarization.library.utility.plot_utils import plot_and_save_history
from keras_text_summarization.library.seq2seq import Seq2SeqSummarizer
from keras_text_summarization.library.applications.fake_news_loader import fit_text
import numpy as np
import re

from keras.models import Model
from keras.layers import Embedding, Dense, Input
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras_text_summarization.library.utility.glove_loader import load_glove, GLOVE_EMBEDDING_SIZE

from corus import load_lenta

#### load and preprocess dataset

In [29]:
lenta_path = 'data/1/lenta-ru-news.csv.gz'
num_rows = 1_000

In [30]:
records = load_lenta(lenta_path)
df = pd.DataFrame([[rec.text, rec.title] for _, rec in zip(range(num_rows), records)], columns=['text', 'title'])

df['title'] = df['title'].apply(lambda x: re.sub("\xa0", " ", x))

reg = re.compile('[^а-яА-Я0-9.,_-_ ]')
df['tokenized_text'] = [reg.sub('', text) for text in df.text.apply(lambda x: x.lower())]
df['tokenized_title'] = [reg.sub('', title) for title in df.title.apply(lambda x: x.lower())]


train_input_tokens, test_input_tokens, train_label_tokens, test_label_tokens = train_test_split(df['tokenized_text'].values, df['tokenized_title'].values, test_size=0.1, shuffle=False)

In [ ]:
X_bert 

In [ ]:
!mkdir bert_cache
!mkdir bert_cache/transformer

In [8]:
from transformers import BertTokenizer, TFBertModel
model_name = 'bert-base-multilingual-cased'


cache_dir = "bert_cache/"
cache_dir_models = cache_dir + "bert_models/"
cache_dir_tokenizers = cache_dir + "bert_tokenizers/"
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir=cache_dir_tokenizers)
bert_model = TFBertModel.from_pretrained(model_name, cache_dir=cache_dir_models)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [74]:
bert_tokenized_text = df['tokenized_text'].apply(lambda x: tokenizer.encode(x, max_length=512, truncation=True))
bert_tokenized_title = df['tokenized_title'].apply(lambda x: tokenizer.encode(x, max_length=32, truncation=True))


input_ids = pad_sequences(bert_tokenized_text, maxlen=512, truncating="post", padding="post")
label_ids = pad_sequences(bert_tokenized_title, maxlen=32, truncating="post", padding="post")

bert_text_train, bert_text_test, bert_title_train, bert_title_test = train_test_split(input_ids, label_ids, test_size=0.1)

In [16]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers.recurrent import LSTM

from keras.preprocessing.sequence import pad_sequences



In [64]:
# input_layer = Input(shape=(512), dtype='int64')
# bert = bert_model.bert(input_layer)
# lstm = LSTM(units=250, return_state=True)(bert[0])
# classifier = Dense(units=2000, activation='softmax')(lstm[0])
# model = Model(inputs=input_layer, outputs=classifier)

# model.layers[1].trainable = False
# model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 512, 768), (None, 177853440 
_________________________________________________________________
lstm_3 (LSTM)                [(None, 250), (None, 250) 1019000   
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              502000    
Total params: 179,374,440
Trainable params: 1,521,000
Non-trainable params: 177,853,440
_________________________________________________________________


In [70]:
input_layer = Input(shape=(512), dtype='int64')
bert = bert_model.bert(input_layer)
classifier = Dense(units=119547, activation='softmax')(bert[0])
model = Model(inputs=input_layer, outputs=classifier)
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 512, 768), (None, 177853440 
_________________________________________________________________
dense_6 (Dense)              (None, 512, 119547)       91931643  
Total params: 269,785,083
Trainable params: 91,931,643
Non-trainable params: 177,853,440
_________________________________________________________________


In [71]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'loss'])

In [78]:
bert_text_train.shape

(900, 512)

In [79]:
bert_title_train.shape

(900, 32)

In [ ]:
model(bert_text_train)

In [77]:
model.fit(bert_text_train, bert_title_train, batch_size=32)

ValueError: in user code:

    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/aydar/opt/anaconda3/lib/python3.7/sit

In [24]:
checkpointer = ModelCheckpoint(filepath='checkpoints/' + '/model-{epoch:02d}.hdf5', verbose=1)   

In [ ]:
# model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
#                         validation_data=valid_data_generator.generate(),
#                         validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])

In [ ]:
model.fit()

#### from keras-text-summarization

In [ ]:
from collections import Counter

MAX_INPUT_SEQ_LENGTH = 1500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 5000
MAX_TARGET_VOCAB_SIZE = 2000

In [29]:
# def fit_text(X, Y, input_seq_max_length=None, target_seq_max_length=None):
X = train_input_tokens
Y = train_label_tokens
input_seq_max_length=None
target_seq_max_length=None


if input_seq_max_length is None:
    input_seq_max_length = MAX_INPUT_SEQ_LENGTH
if target_seq_max_length is None:
    target_seq_max_length = MAX_TARGET_SEQ_LENGTH
input_counter = Counter()
target_counter = Counter()
max_input_seq_length = 0
max_target_seq_length = 0

for line in X:
    text = [word.lower() for word in line.split(' ')]
    seq_length = len(text)
    if seq_length > input_seq_max_length:
        text = text[0:input_seq_max_length]
        seq_length = len(text)
    for word in text:
        input_counter[word] += 1
    max_input_seq_length = max(max_input_seq_length, seq_length)

for line in Y:
    line2 = 'START ' + line.lower() + ' END'
    text = [word for word in line2.split(' ')]
    seq_length = len(text)
    if seq_length > target_seq_max_length:
        text = text[0:target_seq_max_length]
        seq_length = len(text)
    for word in text:
        target_counter[word] += 1
        max_target_seq_length = max(max_target_seq_length, seq_length)

input_word2idx = dict()
for idx, word in enumerate(input_counter.most_common(MAX_INPUT_VOCAB_SIZE)):
    input_word2idx[word[0]] = idx + 2
input_word2idx['PAD'] = 0
input_word2idx['UNK'] = 1
input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])

target_word2idx = dict()
for idx, word in enumerate(target_counter.most_common(MAX_TARGET_VOCAB_SIZE)):
    target_word2idx[word[0]] = idx + 1
target_word2idx['UNK'] = 0

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_input_tokens = len(input_word2idx)
num_target_tokens = len(target_word2idx)

config = dict()
config['input_word2idx'] = input_word2idx
config['input_idx2word'] = input_idx2word
config['target_word2idx'] = target_word2idx
config['target_idx2word'] = target_idx2word
config['num_input_tokens'] = num_input_tokens
config['num_target_tokens'] = num_target_tokens
config['max_input_seq_length'] = max_input_seq_length
config['max_target_seq_length'] = max_target_seq_length

# return config

In [ ]:
fit_text

In [7]:
def transform_input_text(texts, max_input_seq_length=1000):
    input_word2idx = {}
    temp = []
    for line in texts:
        x = []
        for word in line.lower().split(' '):
            wid = 1
            if word in input_word2idx:
                wid = input_word2idx[word]
            x.append(wid)
            if len(x) >= max_input_seq_length:
                break
        temp.append(x)
    temp = pad_sequences(temp, maxlen=max_input_seq_length)

    print(temp.shape)
    return temp, input_word2idx

def transform_target_encoding(texts, max_target_seq_length=40):
    temp = []
    for line in texts:
        x = []
        line2 = 'START ' + line.lower() + ' END'
        for word in line2.split(' '):
            x.append(word)
            if len(x) >= max_target_seq_length:
                break
        temp.append(x)

    temp = np.array(temp)
    print(temp.shape)
    return temp

In [14]:
train_input_tokens[0]

'вицепремьер по социальным вопросам татьяна голикова рассказала, в каких регионах россии зафиксирована наиболее высокая смертность от рака, сообщает риа новости. по словам голиковой, чаще всего онкологические заболевания становились причиной смерти в псковской, тверской, тульской и орловской областях, а также в севастополе. вицепремьер напомнила, что главные факторы смертности в россии  рак и болезни системы кровообращения. в начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. по данным росстата, в 2017 году от рака умерли 289 тысяч человек. это на 3,5 процента меньше, чем годом ранее.'

In [18]:
res = transform_input_text(train_input_tokens)

(900, 1000)


In [25]:
res[0][-2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### calculate variables

In [12]:
HIDDEN_UNITS = 100
LOAD_WEIGHTS = False

In [15]:
from collections import Counter

In [32]:
MAX_INPUT_SEQ_LENGTH = 500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 50000
MAX_TARGET_VOCAB_SIZE = 10000

1245.0

In [61]:
num_input_tokens = int(np.median([len(x) for x in train_input_tokens]))
max_input_seq_length = 1500

num_target_tokens = int(np.median([len(x) for x in train_label_tokens]))
max_output_seq_length = 50

In [58]:
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=num_input_tokens, output_dim=HIDDEN_UNITS,
                              input_length=max_input_seq_length, name='encoder_embedding')

encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')

encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [encoder_state_h, encoder_state_c]

In [63]:
decoder_inputs = Input(shape=(None, num_target_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_target_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

if LOAD_WEIGHTS:
    model.load_weights(weight_file_path)
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [67]:
model.fit()

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [ ]:
with open('log.txt', 'a') as log:
    try:
        for cycle_num in range(num_cycles):
            print(cycle_num)

            history = summarizer.fit(train_input_tokens, train_label_tokens, test_input_tokens, test_label_tokens, epochs=num_epochs)


            # log results
            log.write('Epoch number: {}'.format(str((cycle_num+1) * num_epochs )))
            summarizer.model.save_weights('weights_cycle_{}.h5'.format(cycle_num))
            plot_and_save_history(history, summarizer.model_name, '_result_images/plot_seq2seq_cycle_{}.png'.format(cycle_num), metrics={'loss'})

            for i in np.random.permutation(np.arange(len(train_input_tokens)))[0:20]:
                x = train_input_tokens[i]
                actual_headline = train_label_tokens[i]
                headline = summarizer.summarize(x)
                log.write('Generated Headline: ' + str(headline))
                log.write('Original Headline: ' + str(actual_headline))

            log.write('\n\n')
    except KeyboardInterrupt:
        summarizer.model.save_weights('weights_interrupt.h5')